## Lab 8: Isentropic Potential Vorticity
In this week's lab, we will use archived GFS model data and MetPy functions to create our first isentropic potential vorticity analyses.

<br />


### Useful Documentation
1. Xarray open_dataset:  https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html
2. Matplotlib Contour: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html
3. Cartopy Feature: https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html
4. MetPy Units: https://unidata.github.io/MetPy/latest/tutorials/unit_tutorial.html
5. Datetime: https://docs.python.org/3/library/datetime.html
6. Scipy gaussian_filter: https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter.html
7. MetPy isentropic_interpolation_as_dataset: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.isentropic_interpolation_as_dataset.html#metpy.calc.isentropic_interpolation_as_dataset
8. MetPy potential_temperature: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.potential_temperature.html
9. MetPy potential_vorticity_baroclinic: https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.potential_vorticity_baroclinic.html

<br />

## Tutorial Part I


In the first part of this week's tutorial, I am going to show you the procedure for calculating isentropic potential vorticity from gridded GFS model data..

<br />

1. First, we open a sample dataset with pressure as the vertical coordinate, here representing a GFS analysis from 0000 UTC February 9th, 2023.

In [ ]:
from datetime import datetime
import xarray as xr

time = datetime(2023,2,9,0)
data_location = "/data/AtmSci360/Synp2/Lab_8/"
file_name = f"{time:%m%d%y_%H}_gfs.grib2"

data = xr.open_dataset(f"{data_location}{file_name}", engine='cfgrib', filter_by_keys={'typeOfLevel': 'isobaricInhPa'})
data

<br />

2. Next, we need to convert our vertical coordinate to potential temperature.  To start, we define the desired isentropic level or levels.  At least three isentropic levels are needed for MetPy to be able to compute isentropic potential vorticity (one for the absolute vorticity and two more - one below and one above - for static stability). In the example below, we interpolate data to the 295, 300, and 305 K isentropic surfaces so that we can later compute isentropic potential vorticity on the 300-K surface.

In [ ]:
from metpy.units import units
levels = [295,300,305] * units.kelvin
levels

<br />

3. Now we are ready to use MetPy's isentropic_interpolation_as_dataset function.  To later compute isentropic potential vorticity, we need the u- and v-components of the wind to compute absolute vorticity and pressure to compute static stability (since potential temperature in the numerator of the static stability calculation is defined by the isentropic levels themselves).  Since pressure is automatically calculated by the isentropic_interpolation_as_dataset function, we only need to add the u- (u) and v-components of the wind to this function call.

In [ ]:
import metpy.calc as calc
isen_data = calc.isentropic_interpolation_as_dataset(levels, data.t, data.u, data.v)
isen_data

<br />

4.  Next, we use MetPy's potential_vorticity_baroclinic function to compute isentropic potential vorticity.  The potential_vorticity_baroclinic function requires the following inputs:

    - potential temperature
    - pressure
    - u-component of the full wind
    - v-component of the full wind


In [ ]:
pv = calc.potential_vorticity_baroclinic(isen_data.isentropic_level, isen_data.pressure, isen_data.u, isen_data.v)
pv

<br />

5. Finally, we convert our PV data to PVU by multiplying 10<sup>6</sup>, after which we select a single level of interest.

In [ ]:
pv = pv * (10**6)
leveled_pv = pv.sel(isentropic_level=300)
leveled_pv

<br />

## Tutorial Part II


In this week's lab, we will also create charts of potential temperatures on the dynamic tropopause (a constant-IPV surface).  An example is given below of how to compute the potential temperature on the dynamic tropopause.

<br />

1. The GFS data from NCEP already contain specific fields on the dynamic tropopause (which they define as the 2-PVU surface).  Fortunately for us, these data include all of the variables that we will need to compute the potential temperature on the dynamic tropopause.  To start, we open a sample dataset, here representing a GFS analysis from 0000 UTC February 9th, 2023, selecting only the variables on the dynamic tropopause's defined vertical level (filter_by_keys={'typeOfLevel': 'tropopause'}).


In [ ]:
time = datetime(2023,2,9,0)
data_location = "/data/AtmSci360/Synp2/Lab_8/"
file_name = f"{time:%m%d%y_%H}_gfs.grib2"

data = xr.open_dataset(f"{data_location}{file_name}", engine='cfgrib', filter_by_keys={'typeOfLevel': 'tropopause'})
data

<br />
2.  We then calculate the potential temperature using MetPy's potential temperature function and the data we just opened:

In [ ]:
pt = calc.potential_temperature(data.trpp, data.t)
pt

<br />

### Instructions
So that you can complete this week's lab, I have locally archived the March 10th, 2023 1200 UTC GFS analysis.  The location of these data is provided for you below, as is the filename convention if you wish to use a datetime object to select your time of interest.

Create the following maps and plots for March 10th, 2023 at 1200 UTC:

<ul>
  <li>300 hPa geopotential height, temperature, and wind barbs
  <li>Dynamic tropopause potential temperature and wind barbs
  <li>300 K isentropic potential vorticity and wind barbs
  <li>Skew-T/p-log diagrams for
  <ul>
      <li> KGGW
      <li> KSLE
      <li> KDVN
      </ul>
</ul>

Be sure your Skew-T, ln-p diagrams use observed data from the University of Wyoming.  Finally, be sure your maps follow the "good map" guidelines, and do not forget to import the necessary packages before you start coding.  

In [ ]:
time = 
data_location = "/data/AtmSci360/Synp2/Lab_8/"
file_name = f"{time:%m%d%y_%H}_gfs.grib2"